In [3]:
import pandas as pd
import json

In [49]:
# Create column format dictionary
coldf = pd.read_csv('C:/data/HMDA/HMDA_columns_1981_1989.csv')
cols = json.loads(coldf.to_json(orient='records'))
cols

[{'NAME': 'respondentName', 'START': 0, 'STOP': 28, 'LENGTH': 28},
 {'NAME': 'respondentID', 'START': 28, 'STOP': 36, 'LENGTH': 8},
 {'NAME': 'reportMSA', 'START': 36, 'STOP': 40, 'LENGTH': 4},
 {'NAME': 'censusTract', 'START': 40, 'STOP': 46, 'LENGTH': 6},
 {'NAME': 'state', 'START': 46, 'STOP': 48, 'LENGTH': 2},
 {'NAME': 'county', 'START': 48, 'STOP': 51, 'LENGTH': 3},
 {'NAME': 'supervisoryAgencyCode', 'START': 51, 'STOP': 52, 'LENGTH': 1},
 {'NAME': 'censusValidityFlag', 'START': 52, 'STOP': 54, 'LENGTH': 2},
 {'NAME': 'VA_FHA', 'START': 54, 'STOP': 55, 'LENGTH': 1},
 {'NAME': 'vaNumLoans', 'START': 55, 'STOP': 59, 'LENGTH': 4},
 {'NAME': 'vaTotalLoans', 'START': 59, 'STOP': 68, 'LENGTH': 8},
 {'NAME': 'convLoansFlag', 'START': 68, 'STOP': 69, 'LENGTH': 1},
 {'NAME': 'convNumLoans', 'START': 69, 'STOP': 73, 'LENGTH': 4},
 {'NAME': 'convTotalLoans', 'START': 73, 'STOP': 82, 'LENGTH': 9},
 {'NAME': 'hiFlag', 'START': 82, 'STOP': 83, 'LENGTH': 1},
 {'NAME': 'hiNumLoans', 'START': 83,

In [90]:
#datarange = [x for x in range(1990, 2001, 1)]
datarange = [x for x in range(1981, 1989, 1)]
datarange

[1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988]

In [91]:
first_file = True
for i in datarange:
    print(i)
    folder = "c:/data/HMDA/"
    
    
    filename = folder + "HMD_FACDSB" + str(i)[-2:] + ".txt"
    print(filename)
    
#     with open(filename) as f:
#         line = f.readline()
#         print(line)
   
    if first_file == True:
        df = pd.read_csv(filename, header=None, delimiter="\t", names=["original"])
        first_file = False
    else:
        df = df.append(pd.read_csv(filename, header=None, delimiter="\t", names=["original"]))


1981
c:/data/HMDA/HMD_FACDSB81.txt
1982
c:/data/HMDA/HMD_FACDSB82.txt
1983
c:/data/HMDA/HMD_FACDSB83.txt
1984
c:/data/HMDA/HMD_FACDSB84.txt
1985
c:/data/HMDA/HMD_FACDSB85.txt
1986
c:/data/HMDA/HMD_FACDSB86.txt
1987
c:/data/HMDA/HMD_FACDSB87.txt
1988
c:/data/HMDA/HMD_FACDSB88.txt


In [92]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3979444 entries, 0 to 590436
Data columns (total 1 columns):
original    object
dtypes: object(1)
memory usage: 60.7+ MB


In [93]:
for col in cols:
    print(col['NAME'])
    df[col['NAME']] = df.original.map(lambda x: x[col["START"]:col["STOP"]])

respondentName
respondentID
reportMSA
censusTract
state
county
supervisoryAgencyCode
censusValidityFlag
VA_FHA
vaNumLoans
vaTotalLoans
convLoansFlag
convNumLoans
convTotalLoans
hiFlag
hiNumLoans
hiTotalLoans
multiFlag
multiNumLoans
multiTotalLoans
nonFlag
nonNumLoans
nonTotalLoans
recordQuality


In [94]:
df[["state", "county", "censusTract"]].head()

,state,county,censusTract
0,09,001,060100
1,09,001,060900
2,09,001,061000
3,09,001,061300
4,09,001,071400


In [95]:
DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASS = ''
DB_NAME = 'hmda'

In [ ]:
import pymysql.cursors


# Connect to the database
connection = pymysql.connect(host=DB_HOST,
                             user=DB_USER,
                             password=DB_PASS,
                             db=DB_NAME,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
#     with connection.cursor() as cursor:
#         # Create a new record
    
  
    sql = "INSERT INTO lars1981 ("
    for i in cols:
        sql = sql + i["NAME"] + ", "
        
    sql = sql[:-2]
    sql = sql + ") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
    for i in df.iterrows():
    
        # create tuple of data
        data = [ i[1][x['NAME']].strip() for x in cols]
        data = tuple(data) 

        with connection.cursor() as cursor:
            result_row = cursor.execute(sql, data)
            connection.commit()
            
        if i[0] % 10000 == 0:
            print(i[0])
        
    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT * FROM lars1981"
        cursor.execute(sql)
        result = cursor.fetchone()
        print(result)
finally:
    connection.close()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000

In [69]:
result_row

1